In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import numpy as np
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [25]:
df = pd.read_json('../../data/cryptopunk_txn_history-2021-10-07.jsonl', lines=True)
print("DF Shape:", df.shape)
print("Column Names:", df.columns)
print("Unique Transaction Types:", df.txn_type.unique())
df

DF Shape: (167492, 12)
Column Names: Index(['txn_type', 'from', 'to', 'date', 'timestamp', 'source', 'eth',
       'punk_id', 'from_wallet_address', 'to_wallet_address', 'type',
       'accessories'],
      dtype='object')
Unique Transaction Types: ['Bid Withdrawn' 'Bid' 'Sold' 'Offered' 'Transfer' 'Claimed'
 'Offer Withdrawn' '(Wrap)' '(Unwrap)']


,txn_type,from,to,date,timestamp,source,eth,punk_id,from_wallet_address,to_wallet_address,type,accessories
0,Bid Withdrawn,0xe73a1d,,2021-09-04,2021-09-04,larvalabs,321.00,0,None,None,[Female],"[Green Eye Shadow, Earring, Blonde Bob]"
1,Bid,0xe73a1d,,2021-09-01,2021-09-01,larvalabs,321.00,0,None,None,[Female],"[Green Eye Shadow, Earring, Blonde Bob]"
2,Bid Withdrawn,0x2e5e22,,2021-09-01,2021-09-01,larvalabs,320.00,0,None,None,[Female],"[Green Eye Shadow, Earring, Blonde Bob]"
3,Bid,0x2e5e22,,2021-09-01,2021-09-01,larvalabs,320.00,0,None,None,[Female],"[Green Eye Shadow, Earring, Blonde Bob]"
4,Bid Withdrawn,0x2e5e22,,2021-09-01,2021-09-01,larvalabs,263.00,0,None,None,[Female],"[Green Eye Shadow, Earring, Blonde Bob]"
...,...,...,...,...,...,...,...,...,...,...,...,...
167487,Bid,0xa0a59c,,2017-07-03,2017-07-03,larvalabs,0.25,9999,None,None,[Female],"[Mohawk, Nerd Glasses]"
167488,Bid,0x717403,,2017-06-26,2017-06-26,larvalabs,0.20,9999,None,None,[Female],"[Mohawk, Nerd Glasses]"
167489,Bid,ddaavvee,,2017-06-23,2017-06-23,larvalabs,0.20,9999,None,None,[Female],"[Mohawk, Nerd Glasses]"
167490,Bid,0x5b098b,,2017-06-23,2017-06-23,larvalabs,0.15,9999,None,None,[Female],"[Mohawk, Nerd Glasses]"


In [26]:
df["accessory_count"] = df["accessories"].str.len()
# Get unique punk ids
df_claimed = df[df.txn_type == 'Claimed']
df_claimed.head()

,txn_type,from,to,date,timestamp,source,eth,punk_id,from_wallet_address,to_wallet_address,type,accessories,accessory_count
40,Claimed,,0xc352b5,2017-06-23,2017-06-23,larvalabs,NaN,0,None,None,[Female],"[Green Eye Shadow, Earring, Blonde Bob]",3
103,Claimed,,0xc352b5,2017-06-23,2017-06-23,larvalabs,NaN,1,None,None,[Male],"[Smile, Mohawk]",2
133,Claimed,,0xc352b5,2017-06-23,2017-06-23,larvalabs,NaN,2,None,None,[Female],[Wild Hair],1
150,Claimed,,0xc352b5,2017-06-23,2017-06-23,larvalabs,NaN,3,None,None,[Male],"[Wild Hair, Nerd Glasses, Pipe]",3
162,Claimed,,0xc352b5,2017-06-23,2017-06-23,larvalabs,NaN,4,None,None,[Male],"[Big Shades, Wild Hair, Earring, Goat]",4


In [55]:
#df_sold = df[(df.txn_type == 'Sold') | (df.txn_type == 'Claimed')]
df_sold = df[(df.txn_type == 'Sold')]
df_sold.head()

,txn_type,from,to,date,timestamp,source,eth,punk_id,from_wallet_address,to_wallet_address,type,accessories,accessory_count
22,Sold,0xf5099e,0xe08c32,2018-11-30,2018-11-30,larvalabs,25.00,0,None,None,[Female],"[Green Eye Shadow, Earring, Blonde Bob]",3
30,Sold,0x00d7c9,0xa0a59c,2017-07-07,2017-07-07,larvalabs,1.60,0,None,None,[Female],"[Green Eye Shadow, Earring, Blonde Bob]",3
37,Sold,0xc352b5,0x00d7c9,2017-06-23,2017-06-23,larvalabs,0.98,0,None,None,[Female],"[Green Eye Shadow, Earring, Blonde Bob]",3
52,Sold,EliteCat…,0xcf6165,2020-11-30,2020-11-30,larvalabs,60.00,1,None,None,[Male],"[Smile, Mohawk]",2
78,Sold,0xf5099e,GoWest23,2019-04-06,2019-04-06,larvalabs,31.00,1,None,None,[Male],"[Smile, Mohawk]",2


In [56]:
# Find the mean sale price for a cryptopunk per ID
mean_sale = df_sold.groupby('punk_id')[["eth"]].mean().reset_index()
mean_sale = mean_sale.rename(columns={"eth": "mean_eth"})
mean_sale.mean_eth = mean_sale.mean_eth.fillna(0)
mean_sale

,punk_id,mean_eth
0,0,9.193333
1,1,30.473333
2,14,1.750000
3,33,0.510000
4,53,57.330000
...,...,...
6388,9982,142.000000
6389,9989,0.300000
6390,9990,12.255000
6391,9997,79.495000


In [57]:
# merge dataframes on punk_id
with_avg_sale = pd.merge(df_sold, mean_sale, on="punk_id")
df_punk_claimed = pd.merge(df_claimed, mean_sale, on="punk_id")

with_avg_sale.head()
df_punk_claimed.head()

,txn_type,from,to,date,timestamp,source,eth,punk_id,from_wallet_address,to_wallet_address,type,accessories,accessory_count,mean_eth
0,Claimed,,0xc352b5,2017-06-23,2017-06-23,larvalabs,NaN,0,None,None,[Female],"[Green Eye Shadow, Earring, Blonde Bob]",3,9.193333
1,Claimed,,0xc352b5,2017-06-23,2017-06-23,larvalabs,NaN,1,None,None,[Male],"[Smile, Mohawk]",2,30.473333
2,Claimed,,0xc352b5,2017-06-23,2017-06-23,larvalabs,NaN,14,None,None,[Female],"[Pipe, Pilot Helmet, Hot Lipstick]",3,1.750000
3,Claimed,,0xc352b5,2017-06-23,2017-06-23,larvalabs,NaN,33,None,None,[Male],[Peak Spike],1,0.510000
4,Claimed,,0xc352b5,2017-06-23,2017-06-23,larvalabs,NaN,53,None,None,[Female],"[Big Shades, Straight Hair]",2,57.330000


In [58]:
# Conceptualize the "accessories" column as a 2D array
# Reducing its dimensions from 2 to 1 would allow us to apply value_counts
def to_1D(series):
    return pd.Series([x for _list in series for x in _list])

# Keep one unique punk_id column so that we can accurately count total accessories
unique_punk_id = df.drop_duplicates(subset=['punk_id'])

type_counts = to_1D(unique_punk_id["accessories"])
acc_type_counts = pd.DataFrame({'index':type_counts.index, 'accessory_type':type_counts.values})

print("Total Number of Accessories (shoud match above):", acc_type_counts.shape[0])
acc_type_counts.head()

Total Number of Accessories (shoud match above): 27539


,index,accessory_type
0,0,Green Eye Shadow
1,1,Earring
2,2,Blonde Bob
3,3,Smile
4,4,Mohawk


In [59]:
def under_100(row):
    val = 0
    if row < 100:
        val = 1
    return val

# Create dataframe that has accessory type and the count for all NFTs
acc_counts = pd.DataFrame(to_1D(unique_punk_id["accessories"]).value_counts()).reset_index()
acc_counts = acc_counts.rename(columns={0:'acc_count', 'index':'accessory_type'})
acc_counts = acc_counts.sort_values(by=['acc_count'], ascending=True)
acc_counts['percent_count'] = round(100 * (acc_counts['acc_count'] / acc_counts['acc_count'].sum()), 4)
print(acc_counts)

     accessory_type  acc_count  percent_count
86           Beanie         44         0.1598
85           Choker         48         0.1743
84     Pilot Helmet         54         0.1961
83            Tiara         55         0.1997
82      Orange Side         68         0.2469
..              ...        ...            ...
4              Mole        644         2.3385
3   Purple Lipstick        655         2.3784
2      Hot Lipstick        696         2.5273
1         Cigarette        961         3.4896
0           Earring       2459         8.9292

[87 rows x 3 columns]


In [60]:
acc_counts_dict = dict(zip(acc_counts.accessory_type, acc_counts.acc_count))
acc_pct_dict = dict(zip(acc_counts.accessory_type, acc_counts.percent_count))

print(acc_counts_dict)
print(acc_pct_dict)

{'Beanie': 44, 'Choker': 48, 'Pilot Helmet': 54, 'Tiara': 55, 'Orange Side': 68, 'Buck Teeth': 78, 'Welding Goggles': 86, 'Pigtails': 94, 'Pink With Hat': 95, 'Top Hat': 115, 'Spots': 124, 'Rosy Cheeks': 128, 'Blonde Short': 129, 'Wild White Hair': 136, 'Cowboy Hat': 142, 'Wild Blonde': 144, 'Straight Hair Blonde': 144, 'Big Beard': 146, 'Red Mohawk': 147, 'Vampire Hair': 147, 'Half Shaved': 147, 'Blonde Bob': 147, 'Clown Hair Green': 148, 'Straight Hair Dark': 148, 'Straight Hair': 151, 'Silver Chain': 156, 'Dark Hair': 157, 'Purple Hair': 165, 'Gold Chain': 169, 'Medical Mask': 175, 'Tassle Hat': 178, 'Fedora': 186, 'Police Cap': 203, 'Clown Nose': 212, 'Smile': 238, 'Cap Forward': 254, 'Hoodie': 259, 'Front Beard Dark': 260, 'Frown': 261, 'Purple Eye Shadow': 262, 'Handlebars': 263, 'Blue Eye Shadow': 266, 'Green Eye Shadow': 271, 'Vape': 272, 'Front Beard': 273, 'Chinstrap': 282, 'Luxurious Beard': 286, '3D Glasses': 286, 'Mustache': 288, 'Normal Beard Black': 289, 'Normal Beard': 

In [61]:
def create_count_list(row):
    value_list = []
    for acc in row:
        value_list.append(acc_counts_dict[acc])
    
    return value_list

def create_count(row):
    if row != '':
        value = acc_counts_dict[row]
    else:
        value = None
        
    return value

def create_pct_list(row):
    value_list = []
    for acc in row:
        value_list.append(acc_pct_dict[acc])
    
    return value_list

def remove_list(row):
    value = row[0]
    return value

def remove_list_acc(row):
    value = ','.join(row)
    return value

In [75]:
df["accessory_count"] = df["accessories"].str.len()
# Get unique punk ids
df_claimed = df[df.txn_type == 'Claimed']
df_claimed.head()

#df_sold = df[(df.txn_type == 'Sold') | (df.txn_type == 'Claimed')] # uncomment this to include punks that were not sold
df_sold = df[(df.txn_type == 'Sold')]
df_sold.head()

# Find the mean sale price for a cryptopunk per ID
mean_sale = df_sold.groupby('punk_id')[["eth"]].mean().reset_index()
mean_sale = mean_sale.rename(columns={"eth": "mean_eth"})
mean_sale.mean_eth = mean_sale.mean_eth.fillna(0)
mean_sale

# merge dataframe to add in average sale price, on punk_id
with_avg_sale = pd.merge(df_sold, mean_sale, on="punk_id")
df_punk_claimed = pd.merge(df_claimed, mean_sale, on="punk_id")

# Format the type and accessory columns (remove the brackets)
#with_avg_sale['accessories_percents'] = with_avg_sale.accessories.apply(create_pct_list)
with_avg_sale['type_formatted'] = with_avg_sale['type'].apply(remove_list)
with_avg_sale['accessories_formatted'] = with_avg_sale.accessories.apply(remove_list_acc)
df_punk_claimed['type_formatted'] = df_punk_claimed['type'].apply(remove_list)
df_punk_claimed['accessories_formatted'] = df_punk_claimed.accessories.apply(remove_list_acc)

# Create the top 20 sales dataframe BEFORE the split accessories
df_sales = with_avg_sale.copy()
df_sales_top20 = df_sales[df_sales["mean_eth"] > 45]

# Split the accessories and assign the values
with_avg_sale['accessories_split'] = with_avg_sale['accessories']
with_avg_sale = with_avg_sale.explode('accessories_split').fillna('')
with_avg_sale['accessories_values'] = with_avg_sale.accessories_split.apply(create_count)

df_punk_claimed['accessories_split'] = df_punk_claimed['accessories']
df_punk_claimed = df_punk_claimed.explode('accessories_split').fillna('')
df_punk_claimed['accessories_values'] = df_punk_claimed.accessories_split.apply(create_count)

# Assign the type values
type_value = df_punk_claimed.groupby(by=['type_formatted'], as_index=False)[['type']].count()
type_value = type_value.rename(columns={'type': 'type_value'})

# Merge the claimed dataframe with the type values (Only 10,000 rows)
df_punks = pd.merge(df_punk_claimed, type_value, on="type_formatted")

df_punks

,txn_type,from,to,date,timestamp,source,eth,punk_id,from_wallet_address,to_wallet_address,type,accessories,accessory_count,mean_eth,type_formatted,accessories_formatted,accessories_split,accessories_values,type_value
0,Claimed,,0xc352b5,2017-06-23,2017-06-23,larvalabs,,0,,,[Female],"[Green Eye Shadow, Earring, Blonde Bob]",3,9.193333,Female,"Green Eye Shadow,Earring,Blonde Bob",Green Eye Shadow,271.0,6243
1,Claimed,,0xc352b5,2017-06-23,2017-06-23,larvalabs,,0,,,[Female],"[Green Eye Shadow, Earring, Blonde Bob]",3,9.193333,Female,"Green Eye Shadow,Earring,Blonde Bob",Earring,2459.0,6243
2,Claimed,,0xc352b5,2017-06-23,2017-06-23,larvalabs,,0,,,[Female],"[Green Eye Shadow, Earring, Blonde Bob]",3,9.193333,Female,"Green Eye Shadow,Earring,Blonde Bob",Blonde Bob,147.0,6243
3,Claimed,,0xc352b5,2017-06-23,2017-06-23,larvalabs,,14,,,[Female],"[Pipe, Pilot Helmet, Hot Lipstick]",3,1.750000,Female,"Pipe,Pilot Helmet,Hot Lipstick",Pipe,317.0,6243
4,Claimed,,0xc352b5,2017-06-23,2017-06-23,larvalabs,,14,,,[Female],"[Pipe, Pilot Helmet, Hot Lipstick]",3,1.750000,Female,"Pipe,Pilot Helmet,Hot Lipstick",Pilot Helmet,54.0,6243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17852,Claimed,,straybit…,2017-06-23,2017-06-23,larvalabs,,7523,,,[Alien],"[Earring, Knitted Cap, Medical Mask]",3,8.000000,Alien,"Earring,Knitted Cap,Medical Mask",Knitted Cap,419.0,13
17853,Claimed,,straybit…,2017-06-23,2017-06-23,larvalabs,,7523,,,[Alien],"[Earring, Knitted Cap, Medical Mask]",3,8.000000,Alien,"Earring,Knitted Cap,Medical Mask",Medical Mask,175.0,13
17854,Claimed,,straybit…,2017-06-23,2017-06-23,larvalabs,,7804,,,[Alien],"[Cap Forward, Pipe, Small Shades]",3,2106.000000,Alien,"Cap Forward,Pipe,Small Shades",Cap Forward,254.0,13
17855,Claimed,,straybit…,2017-06-23,2017-06-23,larvalabs,,7804,,,[Alien],"[Cap Forward, Pipe, Small Shades]",3,2106.000000,Alien,"Cap Forward,Pipe,Small Shades",Pipe,317.0,13


In [80]:
df_sales_top20.drop_duplicates(subset="punk_id")

,txn_type,from,to,date,timestamp,source,eth,punk_id,from_wallet_address,to_wallet_address,type,accessories,accessory_count,mean_eth,type_formatted,accessories_formatted
10,Sold,0xb813c6,0xe20388,2021-08-23,2021-08-23 00:00:00,larvalabs,129.99,53,None,None,[Female],"[Big Shades, Straight Hair]",2,57.330000,Female,"Big Shades,Straight Hair"
29,Sold,0x7c0830,0xf605c6,2021-08-27,2021-08-27 00:00:00,larvalabs,87.90,139,None,None,[Male],"[Do-rag, Eye Patch]",2,51.217500,Male,"Do-rag,Eye Patch"
35,Sold,0x09d408,0x9cb8c1,2021-07-04,2021-07-04 00:00:00,larvalabs,180.00,143,None,None,[Male],"[Beanie, Horned Rim Glasses]",2,117.166667,Male,"Beanie,Horned Rim Glasses"
38,Sold,0x1919db,0x53fc45,2021-10-02,2021-10-02 00:06:50,opensea,138.99,150,None,None,[Female],"[Purple Lipstick, Earring]",2,133.495000,Female,"Purple Lipstick,Earring"
46,Sold,0x62bdc6,0xf605c6,2021-08-27,2021-08-27 00:00:00,larvalabs,89.47,161,None,None,[Female],"[Hot Lipstick, Frumpy Hair]",2,51.420000,Female,"Hot Lipstick,Frumpy Hair"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18929,Sold,0x65ba4f,0xf3016b,2021-03-23,2021-03-23 00:00:00,larvalabs,98.00,9953,None,None,[Male],"[Cigarette, Luxurious Beard, Hoodie, Big Shades]",4,93.000000,Male,"Cigarette,Luxurious Beard,Hoodie,Big Shades"
18941,Sold,0x50c946,0xbc6c33,2021-08-23,2021-08-23 00:00:00,larvalabs,69.00,9967,None,None,[Female],"[Knitted Cap, Green Eye Shadow]",2,62.223333,Female,"Knitted Cap,Green Eye Shadow"
18969,Sold,keyboardmonkey3.eth,0x023ef7,2021-09-30,2021-09-30 17:41:52,opensea,135.00,9982,None,None,[Female],"[Stringy Hair, VR, Earring]",3,142.000000,Female,"Stringy Hair,VR,Earring"
18974,Sold,3van.eth,0x0e4437,2021-02-08,2021-02-08 00:00:00,larvalabs,99.99,9997,None,None,[Zombie],"[Front Beard, Cap Forward]",2,79.495000,Zombie,"Front Beard,Cap Forward"


In [81]:
# first we filtered on only those that were sold
# then we filtered on the top x%
# 80 percentile hits a mean_eth of 45.79 --> filter out anything less than 45
mean_sale.describe(percentiles=[0.2,0.4,0.6,0.9, 0.99])

#df_sales_filtered.describe(percentiles=[0.2, 0.4, 0.6, 0.8])

,punk_id,mean_eth
count,6393.000000,6393.000000
mean,5334.129673,29.834085
std,2708.708702,61.512056
min,0.000000,0.000000
20%,2617.200000,1.000000
40%,4345.400000,10.869500
50%,5374.000000,17.225000
60%,6272.200000,23.545692
90%,9058.800000,71.968800
99%,9898.080000,170.000000


In [82]:
df_sales_top1 = df_sales[df_sales["mean_eth"] > 170]

In [83]:
df_sales_top1

,txn_type,from,to,date,timestamp,source,eth,punk_id,from_wallet_address,to_wallet_address,type,accessories,accessory_count,mean_eth,type_formatted,accessories_formatted
339,Sold,0xd1e70f,0x18ee9c,2021-08-06,2021-08-06,larvalabs,375.00,561,None,None,[Male],"[Top Hat, 3D Glasses]",2,198.500000,Male,"Top Hat,3D Glasses"
340,Sold,0xc352b5,KingChad,2020-09-15,2020-09-15,larvalabs,22.00,561,None,None,[Male],"[Top Hat, 3D Glasses]",2,198.500000,Male,"Top Hat,3D Glasses"
893,Sold,disco.et…,daddykal…,2021-07-01,2021-07-01,larvalabs,400.04,1119,None,None,[Zombie],"[Shadow Beard, Do-rag, Eye Patch]",3,300.013333,Zombie,"Shadow Beard,Do-rag,Eye Patch"
894,Sold,Pranksy,0x29b1b2,2021-02-26,2021-02-26,larvalabs,300.00,1119,None,None,[Zombie],"[Shadow Beard, Do-rag, Eye Patch]",3,300.013333,Zombie,"Shadow Beard,Do-rag,Eye Patch"
895,Sold,0x365e35,Pranksy,2021-02-24,2021-02-24,larvalabs,200.00,1119,None,None,[Zombie],"[Shadow Beard, Do-rag, Eye Patch]",3,300.013333,Zombie,"Shadow Beard,Do-rag,Eye Patch"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17728,Sold,Pranksy,0x22eab1,2021-08-28,2021-08-28,larvalabs,499.99,9373,None,None,[Female],"[Black Lipstick, Pilot Helmet, Cigarette]",3,322.495000,Female,"Black Lipstick,Pilot Helmet,Cigarette"
17729,Sold,0x062c54,Pranksy,2021-08-01,2021-08-01,larvalabs,145.00,9373,None,None,[Female],"[Black Lipstick, Pilot Helmet, Cigarette]",3,322.495000,Female,"Black Lipstick,Pilot Helmet,Cigarette"
18581,Sold,rleshner…,0x9c5083,2021-08-02,2021-08-02,larvalabs,300.00,9778,None,None,[Male],"[Cigarette, Hoodie, Big Shades]",3,300.000000,Male,"Cigarette,Hoodie,Big Shades"
18927,Sold,runhot99…,0x5b54ca,2021-08-30,2021-08-30,larvalabs,388.88,9952,None,None,[Male],"[Hoodie, Normal Beard, Smile]",3,228.190000,Male,"Hoodie,Normal Beard,Smile"


In [68]:
#new = df_punk_claimed.accessories_formatted.str.split(",", n = 7, expand = True)

In [78]:
# save formatted data
#df_punks.to_csv('cryptopunks_dashboard3.csv')

In [77]:
# save data
#with_avg_sale.to_csv('cryptopunks_dashboard.csv')

In [78]:
#acc_counts.to_csv('cryptopunks_acc_counts_dashboard.csv')